In [1]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style

# Preprocesado y modelado
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders
from sklearn.preprocessing import OrdinalEncoder
import missingno
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [2]:
import pandas as pd

# Cargamos el archivo csv con los datos

df_central = pd.read_csv("C:/Users/Windows 10/Desktop/MASTER DATASCIENCE/TFM/df_central_2_1.csv")

df_central.head()


,Unnamed: 0,Fecha,Period,PREVISION,E_SIMEL,DESVIO,f_PREV_HIGH,f_PREV_LOW,f_RUN,Dia_Semana,Es_fin_semana,Año,Mes,Día
0,0,2021-01-01,1,0.0,0.0,0.0,0,0,0,4,False,2021,1,1
1,1,2021-01-01,2,0.0,0.0,0.0,0,0,0,4,False,2021,1,1
2,2,2021-01-01,3,0.0,0.0,0.0,0,0,0,4,False,2021,1,1
3,3,2021-01-01,4,0.0,0.0,0.0,0,0,0,4,False,2021,1,1
4,4,2021-01-01,5,0.0,0.0,0.0,0,0,0,4,False,2021,1,1


In [3]:
# Eliminamos las columnas innecesarias

df_central = df_central.drop(columns=['Unnamed: 0', 'DESVIO', 'f_PREV_HIGH', 'f_PREV_LOW'])



In [4]:
# Convertimos la columna 'Fecha' a Datetime para hacer las división en dos dfs

df_central['Fecha'] = pd.to_datetime(df_central['Fecha'])

# Dividimos el DataFrame en dos según las fechas especificas

df_inicio = df_central[df_central['Fecha'] <= '2023-10-31']
df_final = df_central[df_central['Fecha'] >= '2023-11-05']

# Eliminamos la columna 'Fecha' de ambos DataFrames para poder preparar el modelo de Deep Learning

df_inicio = df_inicio.drop(columns=['Fecha'])
df_final = df_final.drop(columns=['Fecha'])



In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Preparamos df_inicio eliminando la variables objetivo del conjunto de entrenamiento (X) y especificamos la variable objetivo del conjunto de prueba (y)

X = df_inicio.drop('E_SIMEL', axis=1)
y = df_inicio['E_SIMEL']

# Dividimos los datos en conjuntos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizamos las características, paso necesario para el modelo de deep learning

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verificamos las dimensiones de los conjuntos de datos para asegurarnos de que todo está correcto

(X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape)


((19764, 8), (4942, 8), (19764,), (4942,))

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Definimos la arquitectura del modelo, con dos capas ocultas de 64 unidades cada una con la función de activación 'relu', que proporciona
# capacidad de modelado no lineal.
# una capa de salida sin función de activación

model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Capa de salida para predicción de un valor continuo
])

# Compilamos el modelo
# utilizamos el optimizador 'adam' con la función de pérdida de Mean Square Error (mse) y la métrica mae

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Sacamos el resumen del modelo

model.summary()

# Entrenamos el modelo con 75 epochs y un batch_size de 32; visualizamos el proceso con verbose = 1

history = model.fit(X_train_scaled, y_train, epochs=75, validation_split=0.2, batch_size=32, verbose=1)

# Evaluamos el conjunto de prueba

test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Loss en el conjunto de prueba: {test_loss}, MAE: {test_mae}")





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4801 (18.75 KB)
Trainable params: 4801 (18.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/75


495/495 [==============================] - 2s 1ms/step - loss: 84.3720 - mae: 4.6760 - val_loss: 41.5248 - val_mae: 3.2022
Epoch 2/75
495/495 [==============================] - 1s 1ms/step - loss: 39.2515 - mae: 3.0336 - val_loss: 38.7564 - val_mae: 2.9450
Epoch 3/75
49

In [7]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split


In [8]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import GradientBoostingRegressor

# Configuramos el imputador MICE con GradientBoostingRegressor y el estimador

mice_imputer = IterativeImputer(estimator=GradientBoostingRegressor(
                                    n_estimators=100,
                                    max_depth=10,
                                    min_samples_split=4,
                                    min_samples_leaf=2,
                                    max_features='sqrt'),
                                    max_iter=10, random_state=42)

# Preparamos los datos para el entrenamiento del imputador MICE eliminando la variable objetivo E_SIMEL

X_mice = df_inicio.drop(columns=['E_SIMEL'])

# Entrenamos el imputador MICE

mice_imputer.fit(X_mice)

# Imprimimos confirmación

"Imputador MICE entrenado con éxito."



'Imputador MICE entrenado con éxito.'

In [9]:
# Seleccionamos las filas del dia 5 del df_final para hacer el tratamiento de la variable f_RUN

df_final_05_11 = df_final[(df_final['Año'] == 2023) & (df_final['Mes'] == 11) & (df_final['Día'] == 5)]

df_final_05_11_para_imputar = df_final_05_11.drop(['E_SIMEL'], axis=1)

df_final_05_11_para_imputar[['f_RUN']] = np.nan  # Primero convertimos la columna f_RUN a NaN

# En la variable creada para la imputación, predecimos los valores para las columnas siguientes

valores_imputados = mice_imputer.transform(df_final_05_11_para_imputar[['Period', 'PREVISION', 'f_RUN', 'Dia_Semana', 'Es_fin_semana', 'Año', 'Mes', 'Día']])

# Realmente solo queremos imputar la columna f_RUN, por lo tanto cogemos solo los valores imputados a la columna en concreto y
# establecemos una condición que si los valores son más grandes que 0.2 establecemos un 1, y si son inferiores establecemos un 0.

valores_imputados_f_RUN = np.where(valores_imputados[:, 2]> 0.2, 1, 0)

df_final_05_11.loc[:, 'f_RUN'] = valores_imputados_f_RUN



In [10]:
# Preparamos los datos de df_final_05_11 para la predicción quitando la variable objetivo

X_final_05_11 = df_final_05_11.drop(['E_SIMEL'], axis=1)

# Normalización de los datos

X_final_05_11_scaled = scaler.transform(X_final_05_11)

# Realizamos la predicciones de E_SIMEL con el modelo de deep learning

e_simel_predicciones = model.predict(X_final_05_11_scaled)

e_simel_predicciones = np.maximum(e_simel_predicciones, 0)

# Mostramos los resultados de la predicción

e_simel_predicciones[:25]



1/1 [==============================] - 0s 85ms/step


array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.09970585],
       [0.23204055],
       [0.27280918],
       [0.12085357],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.07039228],
       [6.5706143 ],
       [7.279591  ],
       [7.2673693 ],
       [6.052717  ],
       [4.527865  ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ]], dtype=float32)

In [11]:
# Convertimos el array a una lista para facilitar la asignación a una nueva columna que llamamos 'Prediccion_E_SIMEL'

predicciones_lista = e_simel_predicciones.flatten().tolist()

# Asignar las predicciones a df_final_05_11

df_final_05_11['Prediccion_E_SIMEL'] = predicciones_lista

# Mostramos las primeras filas para verificar

df_final_05_11[['E_SIMEL', 'PREVISION', 'Prediccion_E_SIMEL']].head(25)




C:\Users\Windows 10\AppData\Local\Temp\ipykernel_70736\496075226.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_05_11['Prediccion_E_SIMEL'] = predicciones_lista


,E_SIMEL,PREVISION,Prediccion_E_SIMEL
24802,0.000,0.0,0.000000
24803,0.000,0.0,0.000000
24804,0.000,0.0,0.000000
24805,0.000,0.0,0.000000
24806,0.000,0.0,0.000000
24807,0.000,0.0,0.099706
24808,0.000,0.0,0.232041
24809,0.000,0.0,0.272809
24810,0.000,0.0,0.120854
24811,0.000,0.0,0.000000


In [12]:
# al igual que en todos los casos anteriores, hacemos un sumatorio y porcentaje de desviación para tener una primera idea de como van las predicciones

suma_real_05 = df_final_05_11['E_SIMEL'].sum()
suma_predicha_05 = df_final_05_11['Prediccion_E_SIMEL'].sum()
suma_prevision_05 = df_final_05_11['PREVISION'].sum()


if suma_real_05 != 0:
    desviacion_porcentual = 100 * (suma_predicha_05 - suma_real_05) / suma_real_05
else:
    desviacion_porcentual = float('inf')  # en caso de división por cero, retorna un valor especial para que no nos dé error



if suma_real_05 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_05 - suma_real_05) / suma_real_05
else:
    desviacion_porcentual_prevision = float('inf')  # en caso de división por cero, retorna un valor especial para que no nos dé error

print("Suma real: ", suma_real_05)
print("Suma predicha: ", suma_predicha_05)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_05)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  15.937000000000001
Suma predicha:  32.49395826458931
Desviación porcentual:  103.89005625016821 %
Suma previsión:  0.0
Desviación porcentual:  -100.0 %


In [13]:
# Una vez tenemos la primera predicción, actualizamos df_inicio con los datos del día 5. Así estamos simulando como sería
# el proceso de predicción en tiempo real

datos_dia_5 = df_final[(df_final['Año'] == 2023) & (df_final['Mes'] == 11) & (df_final['Día'] == 5)]
df_inicio_actualizado = pd.concat([df_inicio, datos_dia_5])

# Preparamos df_actualizado con los datos de df_inicio y los datos del día 5 para reentrenar el modelo de deep learning

X_actualizado = df_inicio_actualizado.drop('E_SIMEL', axis=1)
y_actualizado = df_inicio_actualizado['E_SIMEL']


# Normalizamos con fit_transform

X_total_scaled = scaler.fit_transform(X_actualizado)  # Utilizamos fit_transform 

# Y reentrenamos el modelo de deep learning con todos los datos con los parámetros utilizados para el entrenamiento inicial

model.fit(X_total_scaled, y_actualizado, epochs=75, validation_split=0.2, batch_size=32, verbose=1)

# reentrenamos también el modelo imputador con los nuevos datos con todas las variables menos con E_SIMEL

mice_imputer.fit(df_inicio_actualizado[['Period', 'PREVISION', 'f_RUN', 'Dia_Semana', 'Es_fin_semana', 'Año', 'Mes', 'Día']])



Epoch 1/75
619/619 [==============================] - 1s 2ms/step - loss: 28.0296 - mae: 2.4212 - val_loss: 40.4497 - val_mae: 3.3711
Epoch 2/75
619/619 [==============================] - 1s 2ms/step - loss: 27.6672 - mae: 2.4101 - val_loss: 40.7874 - val_mae: 3.3364
Epoch 3/75
619/619 [==============================] - 1s 1ms/step - loss: 27.4614 - mae: 2.3735 - val_loss: 42.6523 - val_mae: 3.3496
Epoch 4/75
619/619 [==============================] - 1s 1ms/step - loss: 27.3850 - mae: 2.4009 - val_loss: 43.3455 - val_mae: 3.5478
Epoch 5/75
619/619 [==============================] - 1s 1ms/step - loss: 27.1732 - mae: 2.3812 - val_loss: 44.6780 - val_mae: 3.3988
Epoch 6/75
619/619 [==============================] - 1s 1ms/step - loss: 27.0080 - mae: 2.3799 - val_loss: 45.0284 - val_mae: 3.6889
Epoch 7/75
619/619 [==============================] - 1s 1ms/step - loss: 26.9944 - mae: 2.3690 - val_loss: 45.5487 - val_mae: 3.5983
Epoch 8/75
619/619 [==============================] - 1s 1ms/s

IterativeImputer(estimator=GradientBoostingRegressor(max_depth=10,
                                                     max_features='sqrt',
                                                     min_samples_leaf=2,
                                                     min_samples_split=4),
                 random_state=42)

In [14]:
import numpy as np

# Seleccionamos las dilas del dia 6 del df_final para hacer el tratamiento de la variable f_RUN

df_final_06_11 = df_final[(df_final['Año'] == 2023) & (df_final['Mes'] == 11) & (df_final['Día'] == 6)]

df_final_06_11_para_imputar = df_final_06_11.drop(['E_SIMEL'], axis=1)

df_final_06_11_para_imputar[['f_RUN']] = np.nan  # Primero convertimos la columna f_RUN a NaN

# En la variable creada para la imputación, predecimos los valores para las columnas siguientes

valores_imputados = mice_imputer.transform(df_final_06_11_para_imputar[['Period', 'PREVISION', 'f_RUN', 'Dia_Semana', 'Es_fin_semana', 'Año', 'Mes', 'Día']])

# Realmente solo queremos imputar la columna f_RUN, por lo tanto cogemos solo los valores imputados a la columna en concreto

valores_imputados_f_RUN = np.where(valores_imputados[:, 2]> 0.2, 1, 0)

df_final_06_11.loc[:, 'f_RUN'] = valores_imputados_f_RUN


# Verificamos que los valores han sido imputados correctamente
# df_final_05_11.head(25)

# Preparar los datos de df_final_05_11 para la predicción
X_final_06_11 = df_final_06_11.drop(['E_SIMEL'], axis=1)

# Normalizar los datos de X_final_05_11 utilizando el mismo scaler que para los datos de entrenamiento
X_final_06_11_scaled = scaler.transform(X_final_06_11)

# Realizar las predicciones de E_SIMEL con el modelo de deep learning
e_simel_predicciones_06 = model.predict(X_final_06_11_scaled)

e_simel_predicciones_06 = np.maximum(e_simel_predicciones_06, 0)

# Mostrar las primeras 5 predicciones
# e_simel_predicciones[:25]

# Convertir el array de predicciones a una lista para facilitar la asignación
predicciones_lista = e_simel_predicciones_06.flatten().tolist()

# Asignar las predicciones a df_final_05_11
df_final_06_11['Prediccion_E_SIMEL'] = predicciones_lista

# Mostrar las primeras filas para verificar
df_final_06_11[['E_SIMEL', 'PREVISION', 'Prediccion_E_SIMEL']].head(25)



1/1 [==============================] - 0s 23ms/step


C:\Users\Windows 10\AppData\Local\Temp\ipykernel_70736\3426074952.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_06_11['Prediccion_E_SIMEL'] = predicciones_lista


,E_SIMEL,PREVISION,Prediccion_E_SIMEL
24826,0.000,0.0,0.000000
24827,0.000,0.0,0.000000
24828,0.000,0.0,0.000000
24829,0.000,0.0,0.000000
24830,0.000,0.0,0.000000
24831,0.000,0.0,0.000000
24832,0.000,0.0,0.000000
24833,0.000,0.0,0.000000
24834,0.000,0.0,0.000000
24835,0.000,0.0,0.000000


In [15]:
suma_real_06 = df_final_06_11['E_SIMEL'].sum()
suma_predicha_06 = df_final_06_11['Prediccion_E_SIMEL'].sum()
suma_prevision_06 = df_final_06_11['PREVISION'].sum()


if suma_real_06 != 0:
    desviacion_porcentual = 100 * (suma_predicha_06 - suma_real_06) / suma_real_06
else:
    desviacion_porcentual = float('inf')  # en caso de división por cero, retorna un valor especial para que no nos dé error



if suma_real_06 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_06 - suma_real_06) / suma_real_06
else:
    desviacion_porcentual_prevision = float('inf')  # en caso de división por cero, retorna un valor especial para que no nos dé error

print("Suma real: ", suma_real_06)
print("Suma predicha: ", suma_predicha_06)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_06)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  58.542
Suma predicha:  87.57659208774567
Desviación porcentual:  49.596173837152236 %
Suma previsión:  159.4
Desviación porcentual:  172.2831471422227 %


In [16]:
# Creamos un función para agilizar el proceso de actualización, reentreno de los modelos, imputación, predicción y cálculo de las métricas

def predecir_y_actualizar_para_un_dia(dia_actual, dia_siguiente,  mes, año, df_inicio_actualizado, df_final, modelo_deep, imputador):
    """
    Función para actualizar el conjunto de entrenamiento con los datos reales de un día específico,
    realizar la imputación para el día siguiente y predecir los valores de E_SIMEL para ese día.

    Args:
    dia_actual (int): Día actual para el que se actualizarán los datos.
    dia_siguiente (int): Datos del día que queremos hacer las imputaciones y la predicción
    mes (int): Mes del día actual.
    ano (int): Año del día actual.
    df_inicio_actualizado (DataFrame): DataFrame actualizado con los datos hasta el día anterior.
    df_final (DataFrame): DataFrame con los datos a predecir.
    modelo_rf (RandomForestRegressor): Modelo de Random Forest entrenado.
    imputador (IterativeImputer): Imputador MICE entrenado.

    Returns:
    DataFrame: DataFrame con las predicciones para el día siguiente.
    DataFrame: DataFrame actualizado con los datos reales del día actual.
    """
    # Actualización de df_actualizado con los datos de dia_actual

    datos_dia_actual = df_final[(df_final['Año'] == año) & (df_final['Mes'] == mes) & (df_final['Día'] == dia_actual)]
    df_inicio_actualizado = pd.concat([df_inicio_actualizado, datos_dia_actual])

    # Reentrenamos los modelos

    # Una vez tenemos la primera predicción, actualizamos df_inicio con los datos del día 5. Así estamos simulando como sería
    # el proceso de predicción en tiempo real



    # Preparar los datos para el modelo de deep learning
    X_actualizado = df_inicio_actualizado.drop('E_SIMEL', axis=1)
    y_actualizado = df_inicio_actualizado['E_SIMEL']

    
    
    # Normalizar las características
    X_actualizado_scaled = scaler.transform(X_actualizado)

    # Reentrenar el modelo de deep learning con todos los datos
    modelo_deep.fit(X_actualizado_scaled, y_actualizado, epochs=75, validation_split=0.2, batch_size=32, verbose=1)

    imputador.fit(df_inicio_actualizado[['Period', 'PREVISION', 'f_RUN', 'Dia_Semana', 'Es_fin_semana', 'Año', 'Mes', 'Día']])

    

    

      # Imputación de valores a la columna f_RUN

    df_dia_siguiente = df_final[(df_final['Año'] == año) & (df_final['Mes'] == mes) & (df_final['Día'] == dia_siguiente)]
    df_dia_siguiente_para_imputar = df_dia_siguiente.drop(['E_SIMEL'], axis=1)
    df_dia_siguiente_para_imputar[['f_RUN']] = np.nan  
    
    valores_imputados = imputador.transform(df_dia_siguiente_para_imputar)
    
    df_dia_siguiente.loc[:, 'f_RUN'] = np.where(valores_imputados[:, 2] > 0.2, 1, 0) 

    
    
    # Preparar los datos de df_final_05_11 para la predicción
    X_prediccion = df_dia_siguiente.drop(['E_SIMEL'], axis=1)

    # Normalizar los datos de X_final_05_11 utilizando el mismo scaler que para los datos de entrenamiento
    X_prediccion_scaled = scaler.transform(X_prediccion)

    # Realizar las predicciones de E_SIMEL con el modelo de deep learning
    predicted_e_simel = model.predict(X_prediccion_scaled)

    predicted_e_simel = np.maximum(predicted_e_simel, 0)

    # Mostrar las primeras 5 predicciones
    # e_simel_predicciones[:25]

    # Convertir el array de predicciones a una lista para facilitar la asignación
    predicciones_lista = predicted_e_simel.flatten().tolist()

    df_predicciones = df_dia_siguiente[['Año', 'Mes', 'Día', 'PREVISION', 'E_SIMEL']].copy()

    # Asignar las predicciones a df_final_05_11
    df_predicciones['Prediccion_E_SIMEL'] = predicciones_lista

    # Mostrar las primeras filas para verificar
    df_predicciones[['E_SIMEL', 'PREVISION', 'Prediccion_E_SIMEL']].head(25)


    

    # Cálculo de las métricas

    mse = mean_squared_error(df_predicciones['E_SIMEL'], df_predicciones['Prediccion_E_SIMEL'])
    r2 = r2_score(df_predicciones['E_SIMEL'], df_predicciones['Prediccion_E_SIMEL'])
    mae = mean_absolute_error(df_predicciones['E_SIMEL'], df_predicciones['Prediccion_E_SIMEL'])


    return df_predicciones, df_inicio_actualizado, mse, r2, mae

In [17]:
# Llamamos a la función
dia_actual = 6    # Actualización de datos con los que reentrenamos los modelos
dia_siguiente = 7    # Preparación de datos para la imputación y predicción
df_predicciones_07_11, df_inicio_actualizado, mse_07_11, r2_07_11, mae_07_11 = predecir_y_actualizar_para_un_dia(dia_actual, dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_07_11, "R²:", r2_07_11, "MAE:", mae_07_11)

Epoch 1/75
619/619 [==============================] - 1s 2ms/step - loss: 22.0962 - mae: 2.2366 - val_loss: 69.2733 - val_mae: 4.5985
Epoch 2/75
619/619 [==============================] - 1s 1ms/step - loss: 22.0963 - mae: 2.2283 - val_loss: 68.6266 - val_mae: 4.6204
Epoch 3/75
619/619 [==============================] - 1s 1ms/step - loss: 22.2297 - mae: 2.2350 - val_loss: 65.2816 - val_mae: 4.4604
Epoch 4/75
619/619 [==============================] - 1s 1ms/step - loss: 22.0281 - mae: 2.2278 - val_loss: 68.1271 - val_mae: 4.5153
Epoch 5/75
619/619 [==============================] - 1s 1ms/step - loss: 21.8273 - mae: 2.2158 - val_loss: 68.3372 - val_mae: 4.5568
Epoch 6/75
619/619 [==============================] - 1s 1ms/step - loss: 22.0642 - mae: 2.2237 - val_loss: 74.5212 - val_mae: 4.6654
Epoch 7/75
619/619 [==============================] - 1s 1ms/step - loss: 21.9455 - mae: 2.2183 - val_loss: 67.1883 - val_mae: 4.5578
Epoch 8/75
619/619 [==============================] - 1s 1ms/s

In [18]:
# Y como en la predicción anterior calculamos los sumatorios y los porcentajes de desviación

suma_real_07_11 = df_predicciones_07_11['E_SIMEL'].sum()
suma_predicha_07_11 = df_predicciones_07_11['Prediccion_E_SIMEL'].sum()
suma_prevision_07_11 = df_predicciones_07_11['PREVISION'].sum()


if suma_real_07_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_07_11 - suma_real_07_11) / suma_real_07_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_07_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_07_11 - suma_real_07_11) / suma_real_07_11
else:
    desviacion_porcentual_prevision = float('inf')  
    

print("Suma real: ", suma_real_07_11)
print("Suma predicha: ", suma_predicha_07_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_07_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  67.97999999999999
Suma predicha:  13.508122086524963
Desviación porcentual:  -80.12927024635927 %
Suma previsión:  24.0
Desviación porcentual:  -64.6954986760812 %


In [19]:
# Llamamos a la función. Actualización de datos reales del día 7 para reentrenar los modelos
# Preparación de los datos del día posterior, en este caso del día 8, para la imputación y predicción

dia_actual = 7
dia_siguiente = 8

df_predicciones_08_11, df_inicio_actualizado, mse_08_11, r2_08_11, mae_08_11 = predecir_y_actualizar_para_un_dia(dia_actual, dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_08_11, "R²:", r2_08_11, "MAE:", mae_08_11)

Epoch 1/75
620/620 [==============================] - 1s 2ms/step - loss: 19.7991 - mae: 2.1116 - val_loss: 79.0149 - val_mae: 5.1278
Epoch 2/75
620/620 [==============================] - 1s 2ms/step - loss: 19.6061 - mae: 2.1073 - val_loss: 81.5929 - val_mae: 5.2723
Epoch 3/75
620/620 [==============================] - 1s 2ms/step - loss: 19.6776 - mae: 2.1182 - val_loss: 75.5619 - val_mae: 4.9359
Epoch 4/75
620/620 [==============================] - 1s 2ms/step - loss: 19.5434 - mae: 2.1064 - val_loss: 79.4953 - val_mae: 5.1842
Epoch 5/75
620/620 [==============================] - 1s 2ms/step - loss: 19.5776 - mae: 2.1011 - val_loss: 81.8439 - val_mae: 5.2555
Epoch 6/75
620/620 [==============================] - 1s 1ms/step - loss: 19.4254 - mae: 2.0867 - val_loss: 78.0406 - val_mae: 5.0659
Epoch 7/75
620/620 [==============================] - 1s 1ms/step - loss: 19.4935 - mae: 2.0834 - val_loss: 79.0402 - val_mae: 5.1444
Epoch 8/75
620/620 [==============================] - 1s 1ms/s

In [20]:
# Sumatorios y porcentajes

suma_real_08_11 = df_predicciones_08_11['E_SIMEL'].sum()
suma_predicha_08_11 = df_predicciones_08_11['Prediccion_E_SIMEL'].sum()
suma_prevision_08_11 = df_predicciones_08_11['PREVISION'].sum()


if suma_real_08_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_08_11 - suma_real_08_11) / suma_real_08_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_08_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_08_11 - suma_real_08_11) / suma_real_08_11
else:
    desviacion_porcentual_prevision = float('inf')  
    

print("Suma real: ", suma_real_08_11)
print("Suma predicha: ", suma_predicha_08_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_08_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  11.296999999999999
Suma predicha:  0.0
Desviación porcentual:  -100.0 %
Suma previsión:  0.0
Desviación porcentual:  -100.0 %


In [21]:
# Seguimos con el mismo proceso anterior. Llamamos a la función con los días específicos que queremos actualizar, imputar y predecir.

dia_actual = 8
dia_siguiente = 9

df_predicciones_09_11, df_inicio_actualizado, mse_09_11, r2_09_11, mae_09_11 = predecir_y_actualizar_para_un_dia(dia_actual, dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_09_11, "R²:", r2_09_11, "MAE:", mae_09_11)

Epoch 1/75
621/621 [==============================] - 1s 1ms/step - loss: 18.0357 - mae: 2.0017 - val_loss: 92.5414 - val_mae: 5.4746
Epoch 2/75
621/621 [==============================] - 1s 1ms/step - loss: 17.9371 - mae: 2.0049 - val_loss: 96.4301 - val_mae: 5.5406
Epoch 3/75
621/621 [==============================] - 1s 1ms/step - loss: 18.0012 - mae: 2.0077 - val_loss: 98.3483 - val_mae: 5.4249
Epoch 4/75
621/621 [==============================] - 1s 1ms/step - loss: 17.9727 - mae: 2.0027 - val_loss: 96.4917 - val_mae: 5.5492
Epoch 5/75
621/621 [==============================] - 1s 2ms/step - loss: 18.0846 - mae: 2.0388 - val_loss: 103.2837 - val_mae: 5.6745
Epoch 6/75
621/621 [==============================] - 1s 1ms/step - loss: 17.9308 - mae: 2.0186 - val_loss: 96.9109 - val_mae: 5.8540
Epoch 7/75
621/621 [==============================] - 1s 1ms/step - loss: 17.7132 - mae: 1.9943 - val_loss: 99.0949 - val_mae: 5.5641
Epoch 8/75
621/621 [==============================] - 1s 1ms/

In [22]:
# Sumatorios y procentajes

suma_real_09_11 = df_predicciones_09_11['E_SIMEL'].sum()
suma_predicha_09_11 = df_predicciones_09_11['Prediccion_E_SIMEL'].sum()
suma_prevision_09_11 = df_predicciones_09_11['PREVISION'].sum()


if suma_real_09_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_09_11 - suma_real_09_11) / suma_real_09_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_09_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_09_11 - suma_real_09_11) / suma_real_09_11
else:
    desviacion_porcentual_prevision = float('inf')  
    

print("Suma real: ", suma_real_09_11)
print("Suma predicha: ", suma_predicha_09_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_09_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  102.43700000000001
Suma predicha:  46.94646996259689
Desviación porcentual:  -54.170397451509814 %
Suma previsión:  127.30000000000001
Desviación porcentual:  24.271503460663627 %


In [23]:
# Llamamos a la función
dia_actual = 9
dia_siguiente = 10

df_predicciones_10_11, df_inicio_actualizado, mse_10_11, r2_10_11, mae_10_11 = predecir_y_actualizar_para_un_dia(dia_actual,dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_10_11, "R²:", r2_10_11, "MAE:", mae_10_11)

Epoch 1/75
621/621 [==============================] - 1s 1ms/step - loss: 17.0617 - mae: 1.9673 - val_loss: 101.3276 - val_mae: 5.7920
Epoch 2/75
621/621 [==============================] - 1s 1ms/step - loss: 16.8681 - mae: 1.9501 - val_loss: 105.3429 - val_mae: 5.9292
Epoch 3/75
621/621 [==============================] - 1s 1ms/step - loss: 16.8524 - mae: 1.9554 - val_loss: 109.0673 - val_mae: 5.8718
Epoch 4/75
621/621 [==============================] - 1s 1ms/step - loss: 16.8630 - mae: 1.9415 - val_loss: 102.8899 - val_mae: 5.8287
Epoch 5/75
621/621 [==============================] - 1s 2ms/step - loss: 16.9958 - mae: 1.9518 - val_loss: 99.7434 - val_mae: 5.6654
Epoch 6/75
621/621 [==============================] - 1s 1ms/step - loss: 16.8683 - mae: 1.9467 - val_loss: 108.4118 - val_mae: 5.9142
Epoch 7/75
621/621 [==============================] - 1s 1ms/step - loss: 16.7882 - mae: 1.9446 - val_loss: 102.7002 - val_mae: 5.6172
Epoch 8/75
621/621 [==============================] - 1s

In [24]:
# Sumatorios y porcentajes

suma_real_10_11 = df_predicciones_10_11['E_SIMEL'].sum()
suma_predicha_10_11 = df_predicciones_10_11['Prediccion_E_SIMEL'].sum()
suma_prevision_10_11 = df_predicciones_10_11['PREVISION'].sum()


if suma_real_10_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_10_11 - suma_real_10_11) / suma_real_10_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_10_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_10_11 - suma_real_10_11) / suma_real_10_11
else:
    desviacion_porcentual_prevision = float('inf')  

print("Suma real: ", suma_real_10_11)
print("Suma predicha: ", suma_predicha_10_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_10_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  58.955
Suma predicha:  26.45813012123108
Desviación porcentual:  -55.121482281009115 %
Suma previsión:  112.6
Desviación porcentual:  90.99313035365958 %


In [25]:
# Llamamos a la función

dia_actual = 10
dia_siguiente = 13

df_predicciones_13_11, df_inicio_actualizado, mse_13_11, r2_13_11, mae_13_11 = predecir_y_actualizar_para_un_dia(dia_actual,dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_13_11, "R²:", r2_13_11, "MAE:", mae_13_11)

Epoch 1/75
622/622 [==============================] - 1s 1ms/step - loss: 15.7733 - mae: 1.8905 - val_loss: 113.0414 - val_mae: 5.9949
Epoch 2/75
622/622 [==============================] - 1s 1ms/step - loss: 15.9748 - mae: 1.9170 - val_loss: 120.1576 - val_mae: 6.3751
Epoch 3/75
622/622 [==============================] - 1s 1ms/step - loss: 15.8270 - mae: 1.8803 - val_loss: 116.9907 - val_mae: 5.9929
Epoch 4/75
622/622 [==============================] - 1s 1ms/step - loss: 15.9930 - mae: 1.9311 - val_loss: 118.5725 - val_mae: 6.1770
Epoch 5/75
622/622 [==============================] - 1s 1ms/step - loss: 15.7689 - mae: 1.8885 - val_loss: 119.1000 - val_mae: 6.3198
Epoch 6/75
622/622 [==============================] - 1s 1ms/step - loss: 15.8730 - mae: 1.9061 - val_loss: 113.7123 - val_mae: 6.2310
Epoch 7/75
622/622 [==============================] - 1s 1ms/step - loss: 15.7139 - mae: 1.8857 - val_loss: 109.8845 - val_mae: 5.9030
Epoch 8/75
622/622 [==============================] - 1

In [26]:
# Sumas y porcentajes

suma_real_13_11 = df_predicciones_13_11['E_SIMEL'].sum()
suma_predicha_13_11 = df_predicciones_13_11['Prediccion_E_SIMEL'].sum()
suma_prevision_13_11 = df_predicciones_13_11['PREVISION'].sum()

if suma_real_13_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_13_11 - suma_real_13_11) / suma_real_13_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_13_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_13_11 - suma_real_13_11) / suma_real_13_11
else:
    desviacion_porcentual_prevision = float('inf')  

print("Suma real: ", suma_real_13_11)
print("Suma predicha: ", suma_predicha_13_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_13_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  107.141
Suma predicha:  0.0
Desviación porcentual:  -100.0 %
Suma previsión:  20.900000000000002
Desviación porcentual:  -80.492995211917 %


In [27]:
# Llamamos a la función
dia_actual = 13
dia_siguiente = 14

df_predicciones_14_11, df_inicio_actualizado, mse_14_11, r2_14_11, mae_14_11 = predecir_y_actualizar_para_un_dia(dia_actual,dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_14_11, "R²:", r2_14_11, "MAE:", mae_14_11)

Epoch 1/75
622/622 [==============================] - 1s 1ms/step - loss: 14.9094 - mae: 1.8360 - val_loss: 133.4017 - val_mae: 6.6930
Epoch 2/75
622/622 [==============================] - 1s 1ms/step - loss: 14.8615 - mae: 1.8360 - val_loss: 130.8438 - val_mae: 6.5471
Epoch 3/75
622/622 [==============================] - 1s 1ms/step - loss: 14.9893 - mae: 1.8604 - val_loss: 125.1303 - val_mae: 6.2818
Epoch 4/75
622/622 [==============================] - 1s 1ms/step - loss: 14.8188 - mae: 1.8412 - val_loss: 130.3037 - val_mae: 6.5094
Epoch 5/75
622/622 [==============================] - 1s 1ms/step - loss: 15.0295 - mae: 1.8468 - val_loss: 136.5609 - val_mae: 6.8880
Epoch 6/75
622/622 [==============================] - 1s 2ms/step - loss: 14.9339 - mae: 1.8568 - val_loss: 130.2867 - val_mae: 6.4838
Epoch 7/75
622/622 [==============================] - 1s 1ms/step - loss: 14.8812 - mae: 1.8594 - val_loss: 123.1039 - val_mae: 6.1942
Epoch 8/75
622/622 [==============================] - 1

In [28]:
# Sumatorios y porcentajes

suma_real_14_11 = df_predicciones_14_11['E_SIMEL'].sum()
suma_predicha_14_11 = df_predicciones_14_11['Prediccion_E_SIMEL'].sum()
suma_prevision_14_11 = df_predicciones_14_11['PREVISION'].sum()

if suma_real_14_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_14_11 - suma_real_14_11) / suma_real_14_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_14_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_14_11 - suma_real_14_11) / suma_real_14_11
else:
    desviacion_porcentual_prevision = float('inf')  

print("Suma real: ", suma_real_14_11)
print("Suma predicha: ", suma_predicha_14_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_14_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  112.33099999999999
Suma predicha:  14.163253784179688
Desviación porcentual:  -87.39150031230943 %
Suma previsión:  120.5
Desviación porcentual:  7.272257880727503 %


In [29]:
# Llamamos a la función

dia_actual = 14
dia_siguiente = 15

df_predicciones_15_11, df_inicio_actualizado, mse_15_11, r2_15_11, mae_15_11 = predecir_y_actualizar_para_un_dia(dia_actual,dia_siguiente, 11, 2023, df_inicio_actualizado, df_final, model, mice_imputer)
print("MSE:", mse_15_11, "R²:", r2_15_11, "MAE:", mae_15_11)

Epoch 1/75
623/623 [==============================] - 1s 1ms/step - loss: 13.9948 - mae: 1.7905 - val_loss: 141.8430 - val_mae: 6.7528
Epoch 2/75
623/623 [==============================] - 1s 1ms/step - loss: 14.0725 - mae: 1.7873 - val_loss: 142.2749 - val_mae: 6.8270
Epoch 3/75
623/623 [==============================] - 1s 1ms/step - loss: 14.1516 - mae: 1.8140 - val_loss: 139.4073 - val_mae: 6.7659
Epoch 4/75
623/623 [==============================] - 1s 1ms/step - loss: 13.9381 - mae: 1.8031 - val_loss: 141.0948 - val_mae: 7.0118
Epoch 5/75
623/623 [==============================] - 1s 1ms/step - loss: 13.8802 - mae: 1.7892 - val_loss: 137.1393 - val_mae: 6.7593
Epoch 6/75
623/623 [==============================] - 1s 1ms/step - loss: 13.7777 - mae: 1.7783 - val_loss: 136.0222 - val_mae: 6.6676
Epoch 7/75
623/623 [==============================] - 1s 1ms/step - loss: 14.0735 - mae: 1.8235 - val_loss: 144.7623 - val_mae: 6.8497
Epoch 8/75
623/623 [==============================] - 1

In [30]:
# Sumas y porcentajes

suma_real_15_11 = df_predicciones_15_11['E_SIMEL'].sum()
suma_predicha_15_11 = df_predicciones_15_11['Prediccion_E_SIMEL'].sum()
suma_prevision_15_11 = df_predicciones_15_11['PREVISION'].sum()

if suma_real_15_11 != 0:
    desviacion_porcentual = 100 * (suma_predicha_15_11 - suma_real_15_11) / suma_real_15_11
else:
    desviacion_porcentual = float('inf')  



if suma_real_15_11 != 0:
    desviacion_porcentual_prevision = 100 * (suma_prevision_15_11 - suma_real_15_11) / suma_real_15_11
else:
    desviacion_porcentual_prevision = float('inf')  

print("Suma real: ", suma_real_15_11)
print("Suma predicha: ", suma_predicha_15_11)
print("Desviación porcentual: ", desviacion_porcentual, "%")
print("Suma previsión: ", suma_prevision_15_11)
print("Desviación porcentual: ", desviacion_porcentual_prevision, "%")

Suma real:  88.333
Suma predicha:  7.528628349304199
Desviación porcentual:  -91.47699234792864 %
Suma previsión:  131.89999999999998
Desviación porcentual:  49.32131819365354 %


In [31]:
# Concatenamos todos los Dataframes que contienen las prediccions, previsiones y datos reales para calcular las méstricas en conjunto

df_predicciones_totales = pd.concat([df_final_05_11, df_final_06_11, df_predicciones_07_11, 
                                     df_predicciones_08_11, df_predicciones_09_11,df_predicciones_10_11, 
                                     df_predicciones_13_11, df_predicciones_14_11, df_predicciones_15_11])

In [32]:
# Cálculo de las métricas para ver si nos indican si mejoran los errores entren la predicción (Prediccion_E_SIMEL), previsión (PREVISION)y producción real(E_SIMEL)

def calcular_metricas(df):
    mae = mean_absolute_error(df['E_SIMEL'], df['Prediccion_E_SIMEL'])
    mse = mean_squared_error(df['E_SIMEL'], df['Prediccion_E_SIMEL'])
    r2 = r2_score(df['E_SIMEL'], df['Prediccion_E_SIMEL'])
    return mae, mse, r2

# Métricas para la predicciones

mae_pred, mse_pred, r2_pred = calcular_metricas(df_predicciones_totales)

# Cambiamos la columna de predicción por la de previsión

df_previsiones = df_predicciones_totales.copy()
df_previsiones['Prediccion_E_SIMEL'] = df_previsiones['PREVISION']

# Métricas para la previsión

mae_prev, mse_prev, r2_prev = calcular_metricas(df_previsiones)

# Visualizamos los resultados

print("MAE Predicciones: ", mae_pred)
print("MSE Predicciones: ", mse_pred)
print("R² Predicciones: ", r2_pred)
print("MAE Previsiones: ", mae_prev)
print("MSE Previsiones: ", mse_prev)
print("R² Previsiones: ", r2_prev)

MAE Predicciones:  2.6799377686568984
MSE Predicciones:  44.57494542511017
R² Predicciones:  0.11868249182433688
MAE Previsiones:  2.3999490740740743
MSE Previsiones:  35.93822944907407
R² Previsiones:  0.2894440918718205


In [33]:
"""
Las métricas visualizadas nos indican que el modelo utilizado mejoran las métricas de las previsiones, vamos 
a ver a continuación si las métricas reflejan esta mejora en predicción respecto la previsión.

"""

'\nLas métricas visualizadas nos indican que el modelo utilizado mejoran las métricas de las previsiones, vamos \na ver a continuación si las métricas reflejan esta mejora en predicción respecto la previsión.\n\n'

In [34]:
# Para saber si llevamos una acumulado positivo de predicciones vs previsiones sobre la producción real,
# calculamos las diferencias absolutas

# Diferencia absoluta entre la predición y la producción real

df_predicciones_totales['diff_pred_real'] = abs(df_predicciones_totales['Prediccion_E_SIMEL'] - df_predicciones_totales['E_SIMEL'])

# Diferencia absoluta entre la previsión y la producción real

df_predicciones_totales['diff_prev_real'] = abs(df_predicciones_totales['PREVISION'] - df_predicciones_totales['E_SIMEL'])


# sumamos todos los valores de las columnas que queremo comparar

suma_e_simel = df_predicciones_totales['E_SIMEL'].sum()
sumas_totales_predicciones = df_predicciones_totales['Prediccion_E_SIMEL'].sum()
sumas_previsiones = df_predicciones_totales['PREVISION'].sum()


# Calculamos las diferencias entre la prediccion y la previsión respecto la producción real E_SIMEL

diferencia_prediccion_vs_produccion_real = abs(sumas_totales_predicciones - suma_e_simel)
diferencia_prevision_vs_produccion_real = abs(sumas_previsiones - suma_e_simel)


# Imprimimos los resultados para poder visualizar si mejoramos las previsiones a lo largo de todas las predicciones.

print(f"Suma de los valores en la columna E_SIMEL: {suma_e_simel}")
print(f"Suma de las predicciones: {sumas_totales_predicciones}")
print(f"Suma de las previsiones : {sumas_previsiones}")


print(f"Diferencia entre predicciones totales y E_SIMEL total: {diferencia_prediccion_vs_produccion_real}")
print(f"Diferencia entre previsiones y E_SIMEL total: {diferencia_prevision_vs_produccion_real}")


# Calculamos la diferencia entre la predicción y la previsión para saber si el modelo de predicción mejora la previsión

diferencia = diferencia_prediccion_vs_produccion_real - diferencia_prevision_vs_produccion_real

if diferencia_prediccion_vs_produccion_real > diferencia_prevision_vs_produccion_real:
    print(f"No mejoramos la predicción respecto la PREVISION real en: {diferencia}, por lo tanto, con este modelo, no estamos mejorando las previsiones.")
else:
    print(f"La predicción es MEJOR que la previsión en: {-diferencia} unidades, por lo tanto, cumplimos nuestro objetivo de mejorar la PREVISIÓN.")

Suma de los valores en la columna E_SIMEL: 622.953
Suma de las predicciones: 228.6751546561718
Suma de las previsiones : 696.6
Diferencia entre predicciones totales y E_SIMEL total: 394.2778453438282
Diferencia entre previsiones y E_SIMEL total: 73.64700000000005
No mejoramos la predicción respecto la PREVISION real en: 320.6308453438281, por lo tanto, con este modelo, no estamos mejorando las previsiones.


In [35]:
suma_prevision = df_final['PREVISION'].sum()
suma_prevision

696.5999999999999

In [36]:
suma_E_SIMEL = df_final['E_SIMEL'].sum()
suma_E_SIMEL

622.953